<a href="https://colab.research.google.com/github/Howl06/practice/blob/main/line_bot_practice_003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息素材準備
handler執行方法設計
啟動應用


'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
'''
引用套件
'''


# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
# 請ngrok 給我一個臨時網址
run_with_ngrok(app)

# 生成實體物件
# 跟line 溝通用
line_bot_api = LineBotApi("vtaRo4pkLqIZjk50gBJ53fFtMpg+8unl4TvLKbyvENmbWkzdriMNIeVm87ZWRBdTeDL7YM4hGte+BrNMmYscJ1WNPvCQbheXUbVevrlKB3zP3CaeRjbPQRPHMsqW+037bLOogqjOrh5ssRj53kqQMQdB04t89/1O/w1cDnyilFU=")
# 收發訊息用
handler = WebhookHandler("de385af73d2121e65ca6fc0490991b02")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
# 讓line丟消息
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    # 取得簽章
    signature = request.headers['X-Line-Signature']

    # get request body as text
    # 取得消息
    body = request.get_data(as_text=True)
    # 打印消息
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    # 把消息交給handler作處理
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

specific_text_message = TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")
# 要是https
analyze_truth_image_message=ImageSendMessage(
    # 回應 用戶圖片
    original_content_url='https://i.imgur.com/M8jnn9B.png',
    # 預覽
    preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
    )
eye_protect_image_message=ImageSendMessage(
    # 回應 用戶圖片
    original_content_url='https://file.jiutuvip.com/2022/1207/20221207050616179.jpg',
    # 預覽
    preview_image_url='https://img.ltn.com.tw/Upload/ent/page/800/2020/07/17/phpu7aHA2.jpg'
    )

In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''
# 設定字典回應消息,可能是資料庫
# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@123":specific_text_message,
  "@分析的本質": analyze_truth_image_message,
  "@給你養養眼": eye_protect_image_message
}

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
# MessageEvent消息事件
# 文字消息 圖片InageMessage 影片VideoMessage等等
def handle_message(event):
    # 當用戶傳來的消息時有@,那我們就從字典裡找回應
    # 若無 則回傳查無此字的文字消息
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質或是@給你養養眼")
        )
# TextSendMessage 商家傳送訊息
# TextMessage 客戶訊息

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://072f-34-135-231-93.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"text","id":"17810580874211","text":"@給你養養眼"},"webhookEventId":"01GVMAJYC9DB8K60YH2GSGKPBR","deliveryContext":{"isRedelivery":false},"timestamp":1678940665941,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"0bf407f54eac486f8e0389cedde040a6","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 04:24:27] "POST / HTTP/1.1" 200 -
